In [1]:
from box import  Item
from container import Container 
import copy,json
from operator import itemgetter
from visualize import plot_3d_rectangle
%matplotlib qt  
from helper import*
occupied_vol = 0
number_boxes = 0
value = 0
result = []
sorted_boxes = []
truck_dimension = [589.8,243.8, 259.1]
VOL= truck_dimension[0]*truck_dimension[1]*truck_dimension[2]
L,W,H = truck_dimension
vol_items = 0
PP = [[0,0,0]]#,[0,W,0]]
removed_PP = []
placed =[]
retry = []
best_point = []
overlaped = False
lcheck,wcheck,hcheck = [False, False,False]
# Plot using matplotlib
x_ticks = 600
y_ticks = 250
z_ticks = 250
step_size =50
effective_dimention ={0:[0,1,2],1:[1,0,2],2:[0,2,1],
                        3:[1,2,0],4:[2,1,0], 5:[2,0,1]}




max_weight=28080
CONT = Container(
    name="ISO-20 feet Dry Container",
    LWH= (589.8,243.8,259.1),
    max_weight=28080,
)
print(CONT.string())
with open('data.json', 'r') as f:
    data = json.load(f)
boxes = [] 

for i in range(len(data)):
    box,r =data[i]
    boxes.append(Item(
        partno = f"Box-{i+1}",
        name= f"C-{box[-1]}",
        weight=box[3],
        LWH = box[0:3],
        rotation = r
    ))    
boxes=sorted(boxes,key=lambda item: (item.get_volume(),item.get_dimention()[0]),reverse=True)#,item[2],item[1]
boxes=sorted(boxes,key=lambda item: (item.get_id().split("C-")[1]),reverse=False)
L,W,H = CONT.get_dimention()
for box in boxes:

    l,w,h = box.get_dimention()
    

    for pos in CONT.PP:  
        
        if pos == [0,0,0]:

            box.onBase = box.set_onBase(True)
            box.set_position(pos)
            box.is_fit =True
            CONT.fit_items.append(box)
            #vol_items += box[4]
            temp_pp = []
            if pos[0]+l < L:
                temp_pp.append( [pos[0]+l,pos[1],pos[2]])

            if pos[1]+w < W:
                temp_pp.append([pos[0],pos[1]+w,pos[2]])

            if pos[1]+h < H:
                temp_pp.append([pos[0],pos[1],pos[2]+h])
            
            CONT.PP.remove(pos)
            best_point.extend([pos])
            CONT.PP.extend(temp_pp)
            box.pps.extend(temp_pp)
            CONT.PP=sorted(CONT.PP,key=lambda point: (point[0],point[2],point[1]), reverse=False)

            break 
                
        if ( L < pos[0]+l or W < pos[1]+w or H < pos[2]+h ): continue 
            
        is_overlap = []
        support_area = []
        is_hanging = []
        temp = {} 

        for current_item in CONT.fit_items:
            box1 = current_item.get_position() + current_item.get_dimention()
            box2  = pos+box.get_dimention()
            is_overlap.append(intersect(box1, box2))
        if True in is_overlap: continue

        box.onBase = box.set_onBase(True) if pos[2]==0 else box.set_onBase(False)
        box.set_position(pos)
        box.is_fit = True
        CONT.fit_items.append(box)
        #vol_items += box[4]
        temp_pp = []
        if pos[0]+l < L and [pos[0]+l,pos[1],pos[2]] not in PP:

            temp_pp.append( [pos[0]+l,pos[1],pos[2]])

        if pos[1]+w < W and [pos[0],pos[1]+w,pos[2]] not in PP:
            temp_pp.append([pos[0],pos[1]+w,pos[2]])

        if pos[2]+h < H and [pos[0],pos[1],pos[2]+h] not in PP:
            temp_pp.append([pos[0],pos[1],pos[2]+h])
        
        CONT.PP.remove(pos)
        best_point.extend([pos])
        CONT.PP.extend(temp_pp)
        box.pps.extend(temp_pp)
        CONT.PP=sorted(CONT.PP,key=lambda point: (point[0],point[2],point[1]), reverse=False)

    # bx =[]       
    # for item in CONT.fit_items:
    #     bx.extend([item.get_position()+item.get_dimention()+[int(item.get_id().split("C-")[1])]])
    # plot_3d_rectangle(bx, x_ticks, y_ticks, z_ticks, step_size,0)
    for item in CONT.fit_items:
        if item.is_fit == False:

            print(item.get_id())
#Convert the list to JSON
# json_data = json.dumps(best_point, indent=2)

# # Save the JSON data to a file
# with open('best_ppV2.json', 'w') as json_file:
#     json_file.write(json_data)

# json_data = json.dumps(placed, indent=2)

# # Save the JSON data to a file
# with open('PP_resV2.json', 'w') as json_file:
#     json_file.write(json_data)
# input()    


ISO-20 feet Dry Container, ((589.8, 243.8, 259.1), max_weight(28080),Vol(37256828.48)


KeyboardInterrupt: 

In [ ]:
# def rectIntersect(box1, box2, x, y):
#     position,d1 = (box1[0:3],box1[3:6])
#     position2,d2 = (box2[0:3],box2[3:6])

#     cx1 = position[x] + d1[x]/2
#     cy1 = position[y] + d1[y]/2
#     cx2 = position2[x] + d2[x]/2
#     cy2 = position2[y] + d2[y]/2

#     ix = max(cx1, cx2) - min(cx1, cx2)
#     iy = max(cy1, cy2) - min(cy1, cy2)
    
#     print(ix < (d1[x]+d2[x])/2)
#     print(iy < (d1[y]+d2[y])/2 )
#     print(ix < (d1[x]+d2[x])/2 and iy < (d1[y]+d2[y])/2)

#     return ix < (d1[x]+d2[x])/2 and iy < (d1[y]+d2[y])/2


# def intersect(box1, box2):
    
#     print(rectIntersect(box1, box2, 0, 1))
#     print(rectIntersect(box1, box2, 1, 2))
#     print(rectIntersect(box1, box2, 0, 2))
    
#     return (
#         rectIntersect(box1, box2, 0, 1) and
#         rectIntersect(box1, box2, 1, 2) and
#         rectIntersect(box1, box2, 0, 2)
#     )

# box1=[25, 92, 0, 25, 110, 43, 1]
# box2= [50, 92, 0, 55, 81, 92, 1]

# intersect(box1, box2)
PP = set([0,0,0])


In [1]:
import copy,json
from operator import itemgetter
from visualize import plot_3d_rectangle
%matplotlib qt  
from helper import*
occupied_vol = 0
number_boxes = 0
value = 0
result = []
sorted_boxes = []
truck_dimension = [589,238, 238]
VOL= truck_dimension[0]*truck_dimension[1]*truck_dimension[2]
L,W,H = truck_dimension
vol_items = 0
PP = [[0,0,0]]#,[0,W,0]]
removed_PP = []
placed =[]
retry = []
best_point = []
overlaped = False
lcheck,wcheck,hcheck = [False, False,False]
# Plot using matplotlib
x_ticks = 600
y_ticks = 250
z_ticks = 250
step_size =50
effective_dimention ={0:[0,1,2],1:[1,0,2],2:[0,2,1],
                        3:[1,2,0],4:[2,1,0], 5:[2,0,1]}
#* reading data
with open('data2.json', 'r') as f:
    data = json.load(f)

# sorted_boxes.extend(sorted([tup for tup in data if tup[0][-1] in {1}],
#                             key=lambda x: (x[0][3]), reverse=True))

# sorted_boxes.extend(sorted([tup for tup in data if tup[0][-1] in {2}],
#                             key=lambda x:(x[0][3]), reverse=True))
# sorted_boxes.extend(sorted([tup for tup in data if tup[0][-1] in {3}],
#                             key=lambda x: (x[0][3]), reverse=True))
for box in data:#sorted_boxes:
    # box_vol = box[3]
    # box_value = box[4]
    # if r == 0:
    #     l, w, h = box[0:3]
    # elif r == 1:
    #     w, l, h = box[0:3]
    # elif r == 2:
    #     l, h, w = box[0:3]
    # elif r == 3:
    #     w, h, l = box[0:3]
    # elif r == 4:
    #     h, w, l = box[0:3]
    # else:
    #     h, l, w = box[0:3]

    # l,w,h = [box[i] for i in effective_dimention[r]]
    # box[0:3] = l,w,h
    l,w,h = box[0:3]
    for pos in PP:  
        # projections = []
        if pos == [0,0,0]:
            placed.append([pos[0],pos[1],pos[2],l,w,h,box[-1]])
            #placed.extend([pos+[l,w,h,box[-1]]])
            vol_items += box[4]
            if pos[0]+l < L:
                PP.extend([ [pos[0]+l,pos[1],pos[2]]])
            if pos[1]+w < W:
                PP.extend([[pos[0],pos[1]+w,pos[2]]])
            if pos[1]+h < H:
                PP.extend([[pos[0],pos[1],pos[2]+h]])
            PP.remove(pos)
            #plot_3d_rectangle(placed, x_ticks, y_ticks, z_ticks, step_size,0)
            best_point.extend([pos])
            #point[0], point[2],
            PP=sorted(PP,key=lambda point: ( point[0], point[2],point[1]), reverse=False)
            # PP=sorted(PP,key=lambda point: ( point[0], point[2],point[1]), reverse=Falsee)
            break      
        
        if ( L < pos[0]+l or W < pos[1]+w or H < pos[2]+h ):
            continue   
        if vol_items > VOL:
                break
                vol_items += box[4]  
        #temp = sorted(placed,key=lambda point: (point[3], point[5], point[4]), reverse=Falsee)
        is_overlap = []
        support_area = [] 
        for box1 in placed:
            # neb_p = [
            #             [box1[0]+box1[3],box1[1],box1[2]],
            #             [box1[0],box1[1]+box1[4],box1[2]],
            #             [box1[0],box1[1],box1[2]+box1[5]]
            #         ]
            # if pos in neb_p:
            is_overlap.append(intersect(box1[0:6], pos+box[0:3]))
    
            # if pos[2] == 0:pass
            # elif box1[5] == pos[2]:
            #     support_area.append(calculate_overlap_area(box1[0:6], pos+box[0:3]))
  
        if True in is_overlap:# or 0.75 not in support_area: 
            #removed_PP.append(PP.remove(pos))
            is_overlap = []
            support_area =[]
            continue
                # if intersect(box1, pos+box[0:3]):
                #     continue
                # if check_hanging(box1, pos+box[0:3]):
                #     continue
                # else:

        # if pos[2] == 0 or any(x > 0.75 for x in support_area ):pass 
        # elif 0.75 not in support_area:continue
        projections= get_projections(box1[0:6],pos+box[0:3])
        placed.append(pos + [l,w,h,box[-1]])
        if projections:
            for p in projections:
                if p[0]+l < L and p not in PP:
                    PP.extend([p])
                if p[1]+w < W and p not in PP:
                    PP.extend([p])

            if pos[2]+h < H and [pos[0],pos[1],pos[2]+h] not in PP:
                PP.extend([[pos[0],pos[1],pos[2]+h]]) 
            # PP = [item for item in PP if item != pos]
            PP.remove(pos)
            #plot_3d_rectangle(placed, x_ticks, y_ticks, z_ticks, step_size,0)
            best_point.extend([pos])
            PP=sorted(PP,key=lambda point: ( point[0], point[2],point[1]), reverse=False)
            # PP=sorted(PP,key=lambda point: (-point[1], -point[0], point[2]))
            # PP=sorted(PP,key=lambda point: (point[0], point[2], point[1]), reverse=Falsee)
            # PP=sorted(PP,key=lambda point: point[0],reverse=Falsee)
            # PP=sorted(PP,key=lambda point: point[2],reverse=Falsee)
            # PP=sorted(PP,key=lambda point: point[1], reverse=Falsee)
            break
        else:

            if pos[0]+l < L and [pos[0]+l,pos[1],pos[2]] not in PP:
                PP.extend([ [pos[0]+l,pos[1],pos[2]]])
            if pos[1]+w < W and [pos[0],pos[1]+w,pos[2]] not in PP:
                PP.extend([[pos[0],pos[1]+w,pos[2]]])
            if pos[2]+h < H and [pos[0],pos[1],pos[2]+h] not in PP:
                PP.extend([[pos[0],pos[1],pos[2]+h]])

            PP.remove(pos)
            #plot_3d_rectangle(placed, x_ticks, y_ticks, z_ticks, step_size,0)
            best_point.extend([pos])
            PP=sorted(PP,key=lambda point: ( point[0], point[2],point[1]), reverse=False)
            # PP=sorted(PP,key=lambda point: point[0],reverse=False)
            # PP=sorted(PP,key=lambda point: point[2],reverse=False)
            # PP=sorted(PP,key=lambda point: point[1], reverse=False)
            break    
        #break
    plot_3d_rectangle(placed, x_ticks, y_ticks, z_ticks, step_size,0)


{'FBL': [0, 81, 55], 'FBR': [92, 81, 55], 'FTL': [0, 81, 110], 'FTR': [92, 81, 110], 'BBL': [0, 162, 55], 'BBR': [92, 162, 55], 'BTL': [0, 162, 110], 'BTR': [92, 162, 110]}


c:\Users\elahi\Desktop\AIDEAS_Local\CLP\visualize.py:98: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = fig = plt.figure(figsize=(15,15))


{'FBL': [189, 92, 110], 'FBR': [214, 92, 110], 'FTL': [189, 92, 153], 'FTR': [214, 92, 153], 'BBL': [189, 202, 110], 'BBR': [214, 202, 110], 'BTL': [189, 202, 153], 'BTR': [214, 202, 153]}


KeyboardInterrupt: 